In [1]:
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import os

### Load weights

In [ ]:
def build_mobilenetv2_model(input_shape=(224, 224, 3), num_classes=5):
    # Load pre-trained base model (không bao gồm lớp fully connected cuối)
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Đóng băng các lớp của base model (để không huấn luyện lại chúng ban đầu)
    base_model.trainable = False # Quan trọng!

    x = base_model.output
    x = GlobalAveragePooling2D()(x) # Giảm chiều dữ liệu
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x) # Sử dụng sigmoid cho multi-label

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def predict_multi_label(image_path, model, threshold=0.5): # Pass model as argument
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return None, None

    image = cv2.resize(image, (224, 224))  # Resize image to 224x224
    image = image / 255.0  # Normalize the image (match training)
    image = image.astype(np.float32) # Ensure float32 dtype

    image = np.expand_dims(image, axis=0)  # Add batch dimension

    predictions = model.predict(image)[0] # Get the prediction for the single image

    # Define the class labels
    class_labels = ['natural', 'sleepy_eye', 'yawn', 'rub_eye', 'look_away']

    print("--- Predicted Probabilities ---")
    predicted_labels = []
    confidences = {}
    for i, label in enumerate(class_labels):
        confidence = predictions[i]
        confidences[label] = confidence
        print(f"{label}: {confidence:.4f}")
        if confidence >= threshold:
            predicted_labels.append(label)

    # If no label meets the threshold, maybe report the one with highest confidence? (Optional)
    if not predicted_labels:
         highest_confidence_idx = np.argmax(predictions)
         print(f"\nNote: No label reached threshold {threshold}.")
         print(f"Highest confidence label: {class_labels[highest_confidence_idx]} ({predictions[highest_confidence_idx]:.4f})")

    return predicted_labels, confidences # Return list of active labels and all confidences

# --- Example Usage ---
# Make sure you load the *final, best* model first
try:
    # Try loading the fine-tuned model first if it exists
    model_path = './model/mobilenet-v2.h5'
    if not os.path.exists(model_path):
         # Fallback to the potentially non-finetuned one if needed
        model_path = './model/mobilenet-v2.h5'
        if not os.path.exists(model_path):
            # Fallback to loading weights if full model save failed or wasn't run
            model_path = './model/model_finetune_checkpoint.weights.h5' # Or model_head_checkpoint...
            print(f"Loading weights from: {model_path}")
            # Need to rebuild the model structure if loading only weights
            model = build_mobilenetv2_model()
            model.load_weights(model_path)
        else:
            print(f"Loading full model from: {model_path}")
            model = load_model(model_path)
    else:
        print(f"Loading full model from: {model_path}")
        model = load_model(model_path)

    image_path_test = 'image06.png' # Change as needed
    predicted_labels, confidences = predict_multi_label(image_path_test, model, threshold=0.5)

    if predicted_labels is not None:
        print(f"\nPredicted active labels (threshold 0.5): {predicted_labels}")

except Exception as e:
    print(f"Error during model loading or prediction: {e}")
    print("Please ensure the model file exists and training completed successfully.")

Loading full model from: ./model/mobilenet-v2.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
--- Predicted Probabilities ---
natural: 0.0380
sleepy_eye: 0.7714
yawn: 0.0167
rub_eye: 0.0005
look_away: 0.0000

Predicted active labels (threshold 0.5): ['sleepy_eye']


In [4]:
# Make sure the correct model is loaded
# from tensorflow.keras.models import load_model
# model = load_model('sleepiness_detection_model_final.h5') # Load the FINE-TUNED model

def build_mobilenetv2_model(input_shape=(224, 224, 3), num_classes=5):
    # Load pre-trained base model (không bao gồm lớp fully connected cuối)
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Đóng băng các lớp của base model (để không huấn luyện lại chúng ban đầu)
    base_model.trainable = False # Quan trọng!

    x = base_model.output
    x = GlobalAveragePooling2D()(x) # Giảm chiều dữ liệu
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x) # Sử dụng sigmoid cho multi-label

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def predict_multi_label(image_path, model, threshold=0.5): # Pass model as argument
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return None, None

    image = cv2.resize(image, (224, 224))  # Resize image to 224x224
    image = image / 255.0  # Normalize the image (match training)
    image = image.astype(np.float32) # Ensure float32 dtype

    image = np.expand_dims(image, axis=0)  # Add batch dimension

    predictions = model.predict(image)[0] # Get the prediction for the single image

    # Define the class labels
    class_labels = ['natural', 'sleepy_eye', 'yawn', 'rub_eye', 'look_away']

    print("--- Predicted Probabilities ---")
    predicted_labels = []
    confidences = {}
    for i, label in enumerate(class_labels):
        confidence = predictions[i]
        confidences[label] = confidence
        print(f"{label}: {confidence:.4f}")
        if confidence >= threshold:
            predicted_labels.append(label)

    # If no label meets the threshold, maybe report the one with highest confidence? (Optional)
    if not predicted_labels:
         highest_confidence_idx = np.argmax(predictions)
         print(f"\nNote: No label reached threshold {threshold}.")
         print(f"Highest confidence label: {class_labels[highest_confidence_idx]} ({predictions[highest_confidence_idx]:.4f})")

    return predicted_labels, confidences # Return list of active labels and all confidences

# Try loading the fine-tuned model first if it exists
model_path = './new_models/mobilenetv2_model_final.h5'
if not os.path.exists(model_path):
        # Fallback to the potentially non-finetuned one if needed
    model_path = './new_models/mobilenetv2_model_final.h5'
    if not os.path.exists(model_path):
        # Fallback to loading weights if full model save failed or wasn't run
        model_path = './new_models/model_finetune_checkpoint.weights.h5' # Or model_head_checkpoint...
        print(f"Loading weights from: {model_path}")
        # Need to rebuild the model structure if loading only weights
        model = build_mobilenetv2_model()
        model.load_weights(model_path)
    else:
        print(f"Loading full model from: {model_path}")
        model = load_model(model_path)
else:
    print(f"Loading full model from: {model_path}")
    model = load_model(model_path)

Loading full model from: ./new_models/mobilenetv2_model_final.h5


In [16]:
image_path_test = 'image06.png' # Change as needed
predicted_labels, confidences = predict_multi_label(image_path_test, model, threshold=0.4)

if predicted_labels is not None:
    print(f"\nPredicted active labels (threshold 0.5): {predicted_labels}")

1/1 [==============================] - 0s 32ms/step
--- Predicted Probabilities ---
natural: 0.0541
sleepy_eye: 0.8350
yawn: 0.0162
rub_eye: 0.0014
look_away: 0.0003

Predicted active labels (threshold 0.5): ['sleepy_eye']
